In [ ]:
# Set and create directories for the TAO Toolkit experiment
import os

os.environ['PROJECT_DIR']='/dli/task/tao_project'
os.environ['SOURCE_DATA_DIR']='/dli/task/data'
os.environ['DATA_DIR']='/dli/task/tao_project/data'
os.environ['MODELS_DIR']='/dli/task/tao_project/models'
os.environ['SPEC_FILES_DIR']='/dli/task/spec_files'

In [ ]:
# Combining configuration components in separate files and writing into one
!cat $SPEC_FILES_DIR/dataset_config.txt \
     $SPEC_FILES_DIR/augmentation_config.txt \
     $SPEC_FILES_DIR/model_config.txt \
     $SPEC_FILES_DIR/bbox_rasterizer_config.txt \
     $SPEC_FILES_DIR/postprocessing_config.txt \
     $SPEC_FILES_DIR/training_config.txt \
     $SPEC_FILES_DIR/cost_function_config.txt \
     $SPEC_FILES_DIR/evaluation_config.txt \
     > $SPEC_FILES_DIR/combined_training_config.txt
!cat $SPEC_FILES_DIR/combined_training_config.txt

In [ ]:
# Read the config file
!cat $SPEC_FILES_DIR/dataset_config.txt

In [ ]:
# Read the config file
!cat $SPEC_FILES_DIR/augmentation_config.txt

In [ ]:
# Read the config file
!cat $SPEC_FILES_DIR/model_config.txt

In [ ]:
# Read the config file
!cat $SPEC_FILES_DIR/bbox_rasterizer_config.txt

In [ ]:
# Read the config file
!cat $SPEC_FILES_DIR/postprocessing_config.txt

In [ ]:
# Read the config file
!cat $SPEC_FILES_DIR/training_config.txt

In [ ]:
# Read the config file
!cat $SPEC_FILES_DIR/cost_function_config.txt

In [ ]:
# Read the config file
!cat $SPEC_FILES_DIR/evaluation_config.txt

In [ ]:
# Combining configuration components in separate files and writing into one
!cat $SPEC_FILES_DIR/dataset_config.txt \
     $SPEC_FILES_DIR/augmentation_config.txt \
     $SPEC_FILES_DIR/model_config.txt \
     $SPEC_FILES_DIR/bbox_rasterizer_config.txt \
     $SPEC_FILES_DIR/postprocessing_config.txt \
     $SPEC_FILES_DIR/training_config.txt \
     $SPEC_FILES_DIR/cost_function_config.txt \
     $SPEC_FILES_DIR/evaluation_config.txt \
     > $SPEC_FILES_DIR/combined_training_config.txt
!cat $SPEC_FILES_DIR/combined_training_config.txt

In [ ]:
# View train usage
!detectnet_v2 train --help

In [ ]:
# Initiate the training process
!detectnet_v2 train -e $SPEC_FILES_DIR/combined_training_config.txt \
                    -r $MODELS_DIR/resnet18_detector \
                    -k tlt_encode \
                    -n resnet18_detector

In [ ]:
# View saved weights
print('Model for Each Epoch:')
print('---------------------')

!ls -lh $MODELS_DIR/resnet18_detector/weights

In [ ]:
# View evaluate usage
!detectnet_v2 evaluate --help

In [ ]:
# Evaluate the model using the same validation set as training
!detectnet_v2 evaluate -e $SPEC_FILES_DIR/combined_training_config.txt\
                       -m $MODELS_DIR/resnet18_detector/weights/resnet18_detector.hdf5
                       -k tlt_encode

In [ ]:
# Read the config file
!cat $SPEC_FILES_DIR/inferencer_config.txt

In [ ]:
# Read the config file
!cat $SPEC_FILES_DIR/bbox_handler_config.txt

In [ ]:
# Combining configuration components in separate files and writing into one
!cat $SPEC_FILES_DIR/inferencer_config.txt \
     $SPEC_FILES_DIR/bbox_handler_config.txt \
     > $SPEC_FILES_DIR/combined_inference_config.txt
!cat $SPEC_FILES_DIR/combined_inference_config.txt

In [ ]:
# View inference usage
!detectnet_v2 inference --help

In [ ]:
# Perform inference on the validation set
!detectnet_v2 inference -e $SPEC_FILES_DIR/combined_inference_config.txt \
                        -r $PROJECT_DIR/tao_infer_testing \
                        -i $DATA_DIR/training/images \
                        -k tlt_encode

In [ ]:
# Simple grid visualizer
%matplotlib inline
import matplotlib.pyplot as plt
import os
from math import ceil
import random

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['PROJECT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path)]
    for idx, img_path in enumerate(random.sample(a, num_images)):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img)

In [ ]:
# Visualizing the random 12 images
OUTPUT_PATH = 'tao_infer_testing/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 4 # number of columns in the visualizer grid.
IMAGES = 12 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

In [ ]:
# View export usage
!detectnet_v2 export --help

In [ ]:
# Removing a pre-existing copy if there has been any.
!rm -rf $MODELS_DIR/resnet18_detector_unpruned
!mkdir -p $MODELS_DIR/resnet18_detector_unpruned

In [ ]:
# Exporting .hdf5/.tlt model
!detectnet_v2 export -m $MODELS_DIR/resnet18_detector/weights/resnet18_detector.hdf5 \
                     -o $MODELS_DIR/resnet18_detector_unpruned/resnet18_detector \
                     -k tlt_encode \
                     -e $SPEC_FILES_DIR/combined_training_config.txt \
                     --gen_ds_config

In [ ]:
# Read the nvinfer config file using the TrafficCamNet as is
!cat $SPEC_FILES_DIR/pgie_config_trafficcamnet.txt

In [ ]:
# Run the DeepStream Pipeline with the Purpose-built Model as is
%run sample_apps/app_03.py data/sample_30.h264 spec_files/pgie_config_trafficcamnet.txt output.mp4

# Convert the video to a format that is compatible with Jupyter Lab
!ffmpeg -i output.mp4 output_conv.mp4 -y -loglevel quiet

In [ ]:
# Read the nvinfer config file using the custom model
!cat $SPEC_FILES_DIR/pgie_config_resnet18_detector_unpruned.txt

In [ ]:
# Run the DeepStream Pipeline with the Custom Model
%run sample_apps/app_03.py /dli/task/data/sample_30.h264 spec_files/pgie_config_resnet18_detector_unpruned.txt output_resnet18_detector_unpruned.mp4

# Convert the video to a format that is compatible with Jupyter Lab
!ffmpeg -i output_resnet18_detector_unpruned.mp4 output_resnet18_detector_unpruned_conv.mp4 -y -loglevel quiet

In [ ]:
# Compare results
from IPython.display import HTML

HTML("""
<div>
    <video alt="input" width='49%' autoplay>
        <source src="output_conv.mp4" type="video/mp4">
    </video>
    <video alt="output" width='49%' autoplay>
        <source src="output_resnet18_detector_unpruned_conv.mp4" type="video/mp4">
    </video>
</div>
""")